In [1]:
import numpy as np
import pandas as pd
import requests, re
from bs4 import BeautifulSoup
from bs4 import Tag, NavigableString, Comment


# Create a function to parse through web pages

def parse_page(_d):
    data = filter(lambda x:x != '\n', [i for i in _d.find('table', {'id':'Browse'}).td.contents if isinstance(i, str) or i.name == 'a'])
    _next = next(data, None)
    while _next is not None:
        new_d, _n = {'title':_next.text, 'link':_next['href']}, next(data, None)
        if hasattr(_n, 'text'):
            yield new_d
            yield {'title':_n.text, 'link':_n['href'], 'about':next(data)}
        else:
            yield {**new_d, 'about':_n}   
        _next = next(data, None)  
        

# Identify the web page to start at and create a parsed text list with parse_page

d, r = BeautifulSoup(requests.get('https://www.mtsamples.com/site/pages/browse.asp?type=42-Neurology').text, 'html.parser'), []
r.append(list(parse_page(d)))
_c = [i for i in d.find('div', {'class':'Contrast'}).find_all('a') if i.text == '>']


# Use a while loop to continuously iterate over the links as long as the "next" (>) button is present. 
# At each iteration, scrape the next page and extract the headers with the link and description

while _c:
    d = BeautifulSoup(requests.get(f'https://www.mtsamples.com{_c[0]["href"]}').text, 'html.parser')
    r.append(list(parse_page(d)))
    _c = [i for i in d.find('div', {'class':'Contrast'}).find_all('a') if i.text == '>']
                          

In [2]:
# Specify medical specialty of web pages

neurology_web_pages = r
len(neurology_web_pages)

23

In [3]:
neurology_web_pages

[[{'title': 'Acute Intracerebral Hemorrhage',
   'link': '/site/pages/sample.asp?Type=42-Neurology&Sample=497-Acute Intracerebral Hemorrhage',
   'about': 'MRI - Intracerebral hemorrhage (very acute clinical changes occurred immediately prior to scan).'},
  {'title': 'Adult Hydrocephalus',
   'link': '/site/pages/sample.asp?Type=42-Neurology&Sample=1695-Adult Hydrocephalus',
   'about': 'The patient is a 74-year-old woman who presents for neurological consultation for possible adult hydrocephalus.  Mild gait impairment and mild cognitive slowing.'},
  {'title': 'Alzheimer Disease',
   'link': '/site/pages/sample.asp?Type=42-Neurology&Sample=566-Alzheimer Disease',
   'about': 'MRI brain & PET scan - Dementia of Alzheimer type with primary parietooccipital involvement.'},
  {'title': 'Anaplastic Astrocytoma - Letter',
   'link': '/site/pages/sample.asp?Type=42-Neurology&Sample=1249-Anaplastic Astrocytoma - Letter',
   'about': 'Patient seen in Neuro-Oncology Clinic because of increasing

In [4]:
# Create a url list for the medical records within the specified medical specialty 

neurology_url_list = []

for i in neurology_web_pages:
    for j in i:
        neurology_url_list.append('https://www.mtsamples.com' + j['link'])
        
len(neurology_url_list)

223

In [5]:
print(neurology_url_list)

['https://www.mtsamples.com/site/pages/sample.asp?Type=42-Neurology&Sample=497-Acute Intracerebral Hemorrhage', 'https://www.mtsamples.com/site/pages/sample.asp?Type=42-Neurology&Sample=1695-Adult Hydrocephalus', 'https://www.mtsamples.com/site/pages/sample.asp?Type=42-Neurology&Sample=566-Alzheimer Disease', 'https://www.mtsamples.com/site/pages/sample.asp?Type=42-Neurology&Sample=1249-Anaplastic Astrocytoma - Letter', 'https://www.mtsamples.com/site/pages/sample.asp?Type=42-Neurology&Sample=502-Arachnoid Cyst', 'https://www.mtsamples.com/site/pages/sample.asp?Type=42-Neurology&Sample=1674-Arm Pain - Neuro Consult', 'https://www.mtsamples.com/site/pages/sample.asp?Type=42-Neurology&Sample=501-Arnold Chiari II with Syrinx', 'https://www.mtsamples.com/site/pages/sample.asp?Type=42-Neurology&Sample=540-Astrocytoma', 'https://www.mtsamples.com/site/pages/sample.asp?Type=42-Neurology&Sample=489-AVM with Hemorrhage', 'https://www.mtsamples.com/site/pages/sample.asp?Type=42-Neurology&Sample=

In [6]:
def get_medical_notes(url_link):

    global medical_notes
    medical_notes = []
    
    
    # We will loop through each of the url links provided as input for the function

    for i in url_link:

        url = i
        res = requests.get(url)
        res.raise_for_status()
        html = res.text
        soup = BeautifulSoup(html, 'html.parser')
        
        
        # Here we can get page_title from <h1>

        title_el = soup.find('h1')
        page_title = title_el.text.strip()
        first_hr = title_el.find_next_sibling('hr')
        
        
        # Here we can get the page_title description after we see the text 'description'

        description_title = title_el.find_next_sibling('b', text=re.compile('description', flags=re.I))
        description_text_parts = []
        for s in description_title.next_siblings:
            if s is first_hr:
                break
            if isinstance(s, Tag):
                description_text_parts.append(s.text.strip())
            elif isinstance(s, NavigableString):
                description_text_parts.append(str(s).strip())
        description_text = '\n'.join(p for p in description_text_parts if p.strip())

        
        # Titles are all bold and uppercase
        
        titles = [b for b in first_hr.find_next_siblings('b') if b.text.strip().isupper()]
        
        
        # We can find the text between the titles and assign it to the title we see earlier

        docs = []
        
        for t in titles:
            text_parts = []
            for s in t.next_siblings:
                
                # go until next title
                
                if s in titles:
                    break
                if isinstance(s, Comment):
                    continue
                if isinstance(s, Tag):
                    if s.name == 'div':
                        break
                    text_parts.append(s.text.strip())
                elif isinstance(s, NavigableString):
                    text_parts.append(str(s).strip())
                    
            text = '\n'.join(p for p in text_parts if p.strip())
            
            docs.append({
                'title': t.text.strip(),
                'text': text
            })
            
            
        # Clean the text in docs such that we have one full_text document of medical notes
        
        lis = []

        for j in docs:
            for k in j.values():
                lis.append(k.rstrip('\n'))

        full_text = ''.join(lis)
        
        
        # Append to our medical_notes list

        medical_notes.append(full_text)

    len(medical_notes)

In [7]:
# Get and clean all medical notes for the specified medical specialty 

get_medical_notes(neurology_url_list)

neurology_medical_notes = medical_notes
len(neurology_medical_notes)

223

In [8]:
neurology_medical_notes

['CC:Left hand numbness on presentation; then developed lethargy later that day.HX:On the day of presentation, this 72 y/o RHM suddenly developed generalized weakness and lightheadedness, and could not rise from a chair. Four hours later he experienced sudden left hand numbness lasting two hours. There were no other associated symptoms except for the generalized weakness and lightheadedness. He denied vertigo.\nHe had been experiencing falling spells without associated LOC up to several times a month for the past year.MEDS:procardia SR, Lasix, Ecotrin, KCL, Digoxin, Colace, Coumadin.PMH:1)8/92 evaluation for presyncope (Echocardiogram showed: AV fibrosis/calcification, AV stenosis/insufficiency, MV stenosis with annular calcification and regurgitation, moderate TR, Decreased LV systolic function, severe LAE. MRI brain: focal areas of increased T2 signal in the left cerebellum and in the brainstem probably representing microvascular ischemic disease. IVG (MUGA scan)revealed: global hypo

In [9]:
# Return medical notes and the labeled medical specialty as a dataframe

neurology_df = pd.DataFrame({'Medical Specialty': 'Neurology', 'Medical Notes': neurology_medical_notes})
print(len(neurology_df))
neurology_df.head()


223


,Medical Specialty,Medical Notes
0,Neurology,CC:Left hand numbness on presentation; then de...
1,Neurology,REASON FOR VISIT:The patient is a 74-year-old ...
2,Neurology,CC:Memory difficulty.HX:This 64 y/o RHM had ha...
3,Neurology,
4,Neurology,CC:Seizures.HX:The patient was initially evalu...


In [10]:
import pickle

# pickle dataframe as a checkpoint

pickle_out = open("neurology_df.pickle", "wb") # write file to bytes
pickle.dump(neurology_df, pickle_out)
pickle_out.close()